In [13]:
import static_nda as nda
import datajoint as dj
from commons import mice
from pipeline import experiment
import pandas as pd

In [60]:
animal_keys = dj.U('animal_id') & nda.Scan()
surgery_rest = {'surgery_type':'Cranial Window and Headbar',
                'surgery_outcoome':'Survival'}

animal_dicts = []
for animal_key in animal_keys:
    animal_dict = {**animal_key}
    lines,genotypes = (mice.Genotypes & animal_key).fetch('line','genotype',order_by='line')
    animal_dict['genotype_clean'] = ', '.join([' '.join([l,g]) for l,g in zip(lines,genotypes) 
                                               if g != 'negative'])
    animal_dict['genotype_full'] = ', '.join([' '.join([l,g]) for l,g in zip(lines,genotypes)])
    dob_rel = dj.U('animal_id','sex','dob') * (mice.Mice & animal_key)
    
    first_rel = dob_rel.aggr(experiment.Scan,
                           first_scan_ts = 'CAST(min(scan_ts) AS DATE)',
                           keep_all_rows=True)
    inc_rel = dob_rel.aggr(experiment.Scan & nda.Scan,
                           first_inc_scan_ts = 'CAST(min(scan_ts) AS DATE)',
                           keep_all_rows=True)
    surg_rel = (experiment.Surgery & animal_key & surgery_rest).proj(surg_ts = 'date')
    dob_rel = dob_rel * surg_rel * first_rel * inc_rel 
    
    dob_rel = dob_rel.proj('sex','dob','surg_ts','first_scan_ts','first_inc_scan_ts',
                           age_at_surgery = 'DATEDIFF(surg_ts,dob)',
                           age_at_first_scan = 'DATEDIFF(first_scan_ts,dob)',
                           age_at_first_inc_scan = 'DATEDIFF(first_inc_scan_ts,dob)')
    
    animal_dict = {**animal_dict, **dob_rel.fetch(as_dict=True)[0]}
    animal_dicts.append(animal_dict)
animal_df = pd.DataFrame(animal_dicts)

display(animal_df)

print('\n\n\n animal sex distribution')
display(animal_df[['animal_id','sex']].groupby('sex').count())

print('\n\n\n animal genotype distribution')
display(animal_df[['animal_id','genotype_clean']].groupby('genotype_clean').count())
display(animal_df[['animal_id','genotype_full']].groupby('genotype_full').count())




animal_id                                   genotype_clean  \
0      21067  Ai162_GCamp6 heterozygous, SLC-Cre heterozygous   
1      22620  Ai162_GCamp6 heterozygous, SLC-Cre heterozygous   
2      22846  Ai162_GCamp6 heterozygous, SLC-Cre heterozygous   
3      23343  Ai162_GCamp6 heterozygous, SLC-Cre heterozygous   
4      23555  Ai162_GCamp6 heterozygous, SLC-Cre heterozygous   
5      23656  Ai162_GCamp6 heterozygous, SLC-Cre heterozygous   
6      23964  Ai162_GCamp6 heterozygous, SLC-Cre heterozygous   

                                       genotype_full         dob sex  \
0    Ai162_GCamp6 heterozygous, SLC-Cre heterozygous  2018-12-21   F   
1    Ai162_GCamp6 heterozygous, SLC-Cre heterozygous  2019-07-08   M   
2    Ai162_GCamp6 heterozygous, SLC-Cre heterozygous  2019-08-01   M   
3    Ai162_GCamp6 heterozygous, SLC-Cre heterozygous  2019-10-29   M   
4  Ai162_GCamp6 heterozygous, Dlx5-CreER negative...  2019-11-30   F   
5  Ai162_GCamp6 heterozygous, Dlx5-CreER negative...  2019-12-17   F   
6  Ai162_GCamp6 heterozygous, Dlx5-CreER negative...  2020-02-05   F   

   surgery_id     surg_ts first_scan_ts first_inc_scan_ts  age_at_surgery  \
0           1  2019-03-15    2019-03-18        2019-04-02              84   
1           1  2019-09-24    2019-09-24        2019-10-07              78   
2           1  2019-11-04    2019-11-04        2019-11-07              95   
3           1  2019-12-25    2020-01-07        2020-01-15              57   
4           1  2020-03-12    2020-03-12        2020-03-20             103   
5           1  2020-03-19    2020-03-19        2020-04-30              93   
6           1  2020-06-10    2020-06-10        2020-06-29             126   

   age_at_first_scan  age_at_first_inc_scan  
0                 87                    102  
1                 78                     91  
2                 95                     98  
3                 70                     78  
4                103                    111  
5                 93                    135  
6                126                    145




 animal sex distribution


animal_id
sex           
F            4
M            3




 animal genotype distribution


animal_id
genotype_clean                                            
Ai162_GCamp6 heterozygous, SLC-Cre heterozygous          7

animal_id
genotype_full                                                
Ai162_GCamp6 heterozygous, Dlx5-CreER negative,...          3
Ai162_GCamp6 heterozygous, SLC-Cre heterozygous             4